In [1]:
import pandas as pd
pmid_dataset = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/pmid_dataset.csv.gz')

In [7]:
paper_bioentity = pd.read_csv('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_2m/paper_bioentity_95_with_url_deduplicated.csv.gz', dtype=str)

In [21]:
paper_bioentity

,PMID,Mention,Type,prob,EntityId,url
0,5,myohemerythrin,gene,0.973336935043335,NCBIGene3048,https://bioregistry.io/NCBIGene:3048
1,5,hemerythrin,gene,0.9804603457450868,NCBIGene7248,https://bioregistry.io/NCBIGene:7248
2,55,yeast glyceraldehyde-3-phosphate dehydrogenase,gene,0.9855808615684508,NCBIGene26330,https://bioregistry.io/NCBIGene:26330
3,55,yeast,species,0.9947445392608644,NCBITaxon4932,https://bioregistry.io/NCBITaxon:4932
4,55,glyceraldehyde-3-phosphate,drug,0.963684380054474,meshD005986,https://bioregistry.io/mesh:D005986
...,...,...,...,...,...,...
10406111,37748650,SMC5/6 complex,gene,0.9954244494438172,NCBIGene79677,https://bioregistry.io/NCBIGene:79677
10406112,37748650,tBRCT,gene,0.9879411458969116,NCBIGene10716,https://bioregistry.io/NCBIGene:10716
10406113,37748650,phosphoserines,drug,0.9882856607437134,CHEBI15811,https://bioregistry.io/CHEBI:15811
10406114,37748650,tBRCTs,gene,0.9883208870887756,NCBIGene1788,https://bioregistry.io/NCBIGene:1788


In [5]:
import numpy as np
# read the .npz file
tkg_embeddings_all = np.load('/data/jx4237data/Graph-CoT/Pipeline/updated_data/tkg_embeddings_all.npz')

# access the data from the .npz file
embeddings = tkg_embeddings_all['embeddings']
ids = tkg_embeddings_all['ids']

tkg_embeddings_new = np.load('/data/jx4237data/TKG/TKG_JCDL/Bridge2AI_10k/tkg_updated_360k_embeddings_all.npz')
new_embeddings = tkg_embeddings_new['embeddings']
new_ids = tkg_embeddings_new['ids']
embeddings_dict = {id_: embedding for id_, embedding in zip(ids, embeddings)}
new_embeddings_dict = {str(id_): embedding for id_, embedding in zip(new_ids, new_embeddings)}
combined_embeddings_dict = {**embeddings_dict, **new_embeddings_dict}

In [23]:
import pandas as pd
import numpy as np

def cumulative_distribution_top_n(series, n):
    """
    Calculates the cumulative distribution of the top n most frequent values in a pandas Series.

    Args:
        series (pd.Series): The pandas Series containing the data.
        n (int): The number of top values to consider.

    Returns:
        pd.Series: A pandas Series representing the cumulative distribution of the top n values.
    """
    value_counts = series.value_counts()
    top_n_counts = value_counts.head(n)
    cumulative_sum = top_n_counts.cumsum()
    total_count = series.count()
    cumulative_percentage = (cumulative_sum / total_count) * 100
    return cumulative_percentage

top_50_cumulative = cumulative_distribution_top_n(paper_bioentity['EntityId'], 5000)

# Print the result
print(top_50_cumulative)

# Example to get the percentage of the top 50 entities.
if len(top_50_cumulative) > 0:
    print(f"The top {len(top_50_cumulative)} entities account for {top_50_cumulative.iloc[-1]:.2f}% of all entities.")
else:
    print("There are less than 50 unique entities.")

EntityId
NCBITaxon9606             9.722600
meshD009369              12.094551
NCBITaxon10095           12.902211
NCBITaxon10090           13.626669
NCBITaxon10116           14.091905
                           ...    
NCBIGene1647             79.751523
meshD009081_mim252650    79.754291
NCBIGene8692             79.757058
NCBIGene9733             79.759826
CHEBI39949               79.762593
Name: count, Length: 5000, dtype: float64
The top 5000 entities account for 79.76% of all entities.


In [25]:
top_entity_set = set(top_50_cumulative.keys())

In [26]:
paper_bioentity = paper_bioentity[paper_bioentity['EntityId'].isin(top_entity_set)]

In [30]:
entity_pmids = paper_bioentity.groupby('EntityId')['PMID'].apply(list).to_dict()

In [31]:
import numpy as np
from tqdm import tqdm
entity_embeddings = {}
for entityId, pmids in tqdm(entity_pmids.items()):
    embeddings = [combined_embeddings_dict[pmid] for pmid in pmids]
    entity_embeddings[entityId] = np.sum(embeddings, axis=0)

100%|██████████| 5000/5000 [00:35<00:00, 141.22it/s]


In [32]:
index_to_entity = {}
current_index = 223000000000
for dataset in entity_embeddings.keys():
    index_to_entity[current_index] = dataset
    current_index += 1

In [33]:
entity_to_index = {dataset: index for index, dataset in index_to_entity.items()}

In [34]:
import pickle
pickle.dump(entity_embeddings, open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/entity_embeddings.pkl', 'wb'))

In [35]:
pickle.dump(index_to_entity, open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/index_to_entity.pkl', 'wb'))